# Chap1-2: 自动求导
这次课程我们会了解 PyTorch 中的自动求导机制，自动求导是 PyTorch 中非常重要的特性，能够让我们避免手动去计算非常复杂的导数，这能够极大地减少了我们构建模型的时间，这也是其前身 Torch 这个框架所不具备的特性，下面我们通过例子看看 PyTorch 自动求导的独特魅力以及探究自动求导的更多用法。

In [1]:
import torch
from torch.autograd import Variable

## 一. 简单情况下的求导, 对标量结果进行求导

In [20]:
x=Variable(torch.Tensor([2]),requires_grad=True)
y=x+2
z=y**2+3
print(x)
print(z)

tensor([2.], requires_grad=True)
tensor([19.], grad_fn=<AddBackward>)


#### z对x进行求导

In [21]:
z.backward()
print(x.grad)

tensor([8.])


通过上面的一些列操作，我们从 x 得到了最后的结果out，我们可以将其表示为数学公式

$$
z = (x + 2)^2 + 3
$$
那么我们从 z 对 x 求导的结果就是

$$
\frac{\partial z}{\partial x} = 2 (x + 2) = 2 (2 + 2) = 8
$$

#### 再来一个例子

In [16]:
x=Variable(torch.randn(2,3),requires_grad=True)
w=Variable(torch.randn(3,2),requires_grad=True)
y=Variable(torch.randn(2,2),requires_grad=True)
out=torch.mean(y- torch.matmul(x,w))
out.backward()
#得到x的梯度
print(out)
print('对x求导',x.grad)
print('对y求导',y.grad)
print('对w求导',w.grad)

tensor(-0.2902, grad_fn=<MeanBackward1>)
对x求导 tensor([[ 0.1071, -0.2666,  0.5740],
        [ 0.1071, -0.2666,  0.5740]])
对y求导 tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])
对w求导 tensor([[ 0.1787,  0.1787],
        [-0.1115, -0.1115],
        [ 0.1440,  0.1440]])


#### 扩展:
- 矩阵乘法:`torch.matmul(tensor1,tensor2)`, 
- 求均值:`torch.mean(tensor)`

In [12]:
# 矩阵相乘
tensor1=torch.Tensor([[1,2]])
tensor2=torch.Tensor([[2],[3]])
print(tensor1.shape)
print(tensor2.shape)
print(torch.matmul(tensor1,tensor2))

torch.Size([1, 2])
torch.Size([2, 1])
tensor([[8.]])


In [14]:
# 求均值
tensor=torch.Tensor([[1,2,3],[4,5,6]])
print(tensor.shape)
print(torch.mean(tensor,dim=-1))
print(torch.mean(tensor,dim=0))

torch.Size([2, 3])
tensor([2., 5.])
tensor([2.5000, 3.5000, 4.5000])


## 二. 复杂情况下的自动求导, 对向量或矩阵结果自动求导

In [17]:
m=Variable(torch.FloatTensor([[2,3]]),requires_grad=True)
n=Variable(torch.zeros(1,2))
print('原始n:\n',n)
n[0,0]=m[0,0]**2
n[0,1]=m[0,1]**3
print('新n:\n',n)

原始n:
 tensor([[0., 0.]])
新n:
 tensor([[ 4., 27.]], grad_fn=<CopySlices>)


在 PyTorch 中，如果要调用自动求导，需要往backward()中传入**一个参数**，这个参数的形状和 n 一样大，比如是 $(w_0,\ w_1)$那么自动求导的结果就是： 
$$
\frac{\partial n}{\partial m_0} = w_0 \frac{\partial n_0}{\partial m_0} + w_1 \frac{\partial n_1}{\partial m_0}
$$
$$
\frac{\partial n}{\partial m_1} = w_0 \frac{\partial n_0}{\partial m_1} + w_1 \frac{\partial n_1}{\partial m_1}
$$

In [22]:
# 矩阵n对m进行求导,向backward()中传入一个tensor
n.backward(torch.ones_like(n))
print(m.grad)

tensor([[ 4., 27.]])


通过自动求导我们得到了梯度是 4 和 27，我们可以验算一下 $$
\frac{\partial n}{\partial m_0} = w_0 \frac{\partial n_0}{\partial m_0} + w_1 \frac{\partial n_1}{\partial m_0} = 2 m_0 + 0 = 2 \times 2 = 4
$$$$
\frac{\partial n}{\partial m_1} = w_0 \frac{\partial n_0}{\partial m_1} + w_1 \frac{\partial n_1}{\partial m_1} = 0 + 3 m_1^2 = 3 \times 3^2 = 27
$$ 通过验算我们可以得到相同的结果

## 三. 多次自动求导
通过调用 backward 我们可以进行一次自动求导，如果我们再调用一次 backward，会发现程序报错，没有办法再做一次。这是因为 PyTorch 默认做完一次自动求导之后，计算图就被丢弃了，所以两次自动求导需要手动设置一个东西，我们通过下面的小例子来说明。

In [23]:
x=Variable(torch.FloatTensor([3]),requires_grad=True) # 设置retain_graph 为 True 来保留计算图
y=x*2+x**2+3
print(y)

tensor([18.], grad_fn=<AddBackward>)


In [24]:
y.backward(retain_graph=True) #设置retain_graph为True来保留计算图
print(x.grad)

tensor([8.])


In [25]:
y.backward()
print(x.grad)

tensor([16.])


可以发现 x 的梯度变成了 16，因为这里做了两次自动求导，所以讲第一次的梯度 8 和第二次的梯度 8 加起来得到了 16 的结果。

**小练习**

定义

$$
x = 
\left[
\begin{matrix}
x_0 \\
x_1
\end{matrix}
\right] = 
\left[
\begin{matrix}
2 \\
3
\end{matrix}
\right]
$$

$$
k = (k_0,\ k_1) = (x_0^2 + 3 x_1,\ 2 x_0 + x_1^2)
$$

我们希望求得

$$
j = \left[
\begin{matrix}
\frac{\partial k_0}{\partial x_0} & \frac{\partial k_0}{\partial x_1} \\
\frac{\partial k_1}{\partial x_0} & \frac{\partial k_1}{\partial x_1}
\end{matrix}
\right]
$$

参考答案：

$$
\left[
\begin{matrix}
4 & 3 \\
2 & 6 \\
\end{matrix}
\right]
$$

In [39]:
x=Variable(torch.Tensor([2,3]),requires_grad=True)
k=Variable(torch.zeros(2))

k[0]=x[0]**2+3*x[1]
k[1]=x[0]*2+x[1]**2

k.backward(torch.FloatTensor([1, 0]), retain_graph=True) # [1,0]是对应下边的公式
print(x.grad)#对应答案第一行

answer=torch.zeros(2,2) #保存到j
answer[0]=x.grad

tensor([4., 3.])


$$
\frac{\partial k}{\partial x_0} = w_0 \frac{\partial k_0}{\partial x_0} + w_1 \frac{\partial k_1}{\partial x_0}
=1*2x_0+0*2
$$
$$
\frac{\partial k}{\partial x_1} = w_0 \frac{\partial k_0}{\partial x_1} + w_1 \frac{\partial k_1}{\partial x_1}
=1*3+0*2x_1
$$

In [40]:
x.grad.zero_() #inplace操作, 将之前求得的梯度归零
k.backward(torch.FloatTensor([0,1])) #此处的[0,1]对应下边的公式:
j[1]=x.grad.data
print(j)

tensor([[4., 3.],
        [2., 6.]])


$$
\frac{\partial k}{\partial x_0} = w_0 \frac{\partial k_0}{\partial x_0} + w_1 \frac{\partial k_1}{\partial x_0}
=0*2x_0+1*2
$$
$$
\frac{\partial k}{\partial x_1} = w_0 \frac{\partial k_0}{\partial x_1} + w_1 \frac{\partial k_1}{\partial x_1}
=0*3+1*2x_1
$$

### torch语法小结
- `torch.mean(tensor, dim=)`:求均值
- `torch.matmul(tensor1,tensor2)`:求矩阵乘法
- `torch.ones_like(tensor)`:得到全1矩阵,注意下划线\_
- `tensor.zero_()`:将矩阵设为全0矩阵,inplace操作